<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/20231228_diffusers0_25_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# diffusers-0.25.0の機能テスト

T4 GPU以上で動きます

https://github.com/huggingface/diffusers/releases/tag/v0.25.0

https://note.com/o_ob/n/nf7869b5974e4



# aMUSEd

aMUSEd は、MUSE アーキテクチャに基づいた軽量の *TextToImageモデル* です。
aMUSEd は、一度に多くの画像を素早く生成するなど、軽量で高速なモデルを必要とするアプリケーションで特に役立ちます。 aMUSEd は現在研究リリースです。

aMUSEd は、多くの diffusion モデルよりも少ない順方向パスでイメージを生成できる VQVAE トークンベースのトランスフォーマーです。 MUSE とは対照的に、T5-XXL の代わりに小型のテキスト エンコーダー CLIP-L/14 を使用します。 ammused はパラメータ数が少なく、フォワード パス生成プロセスが少ないため、多くの画像を迅速に生成できます。 この利点は、特に大きなバッチサイズで見られます。

## Text-to-image generation

```
import torch
from diffusers import AmusedPipeline

pipe = AmusedPipeline.from_pretrained(
    "amused/amused-512", variant="fp16", torch_dtype=torch.float16
)
pipe.vqvae.to(torch.float32)  # vqvae is producing nans in fp16
pipe = pipe.to("cuda")

prompt = "cowboy"
image = pipe(prompt, generator=torch.manual_seed(8)).images[0]
image.save("text2image_512.png")
Image-to-image generation

import torch
from diffusers import AmusedImg2ImgPipeline
from diffusers.utils import load_image

pipe = AmusedImg2ImgPipeline.from_pretrained(
    "amused/amused-512", variant="fp16", torch_dtype=torch.float16
)
pipe.vqvae.to(torch.float32)  # vqvae is producing nans in fp16
pipe = pipe.to("cuda")

prompt = "apple watercolor"
input_image = (
    load_image(
        "https://raw.githubusercontent.com/huggingface/amused/main/assets/image2image_256_orig.png"
    )
    .resize((512, 512))
    .convert("RGB")
)

image = pipe(prompt, input_image, strength=0.7, generator=torch.manual_seed(3)).images[0]
image.save("image2image_512.png")
```

## Inpainting
```
import torch
from diffusers import AmusedInpaintPipeline
from diffusers.utils import load_image
from PIL import Image

pipe = AmusedInpaintPipeline.from_pretrained(
    "amused/amused-512", variant="fp16", torch_dtype=torch.float16
)
pipe.vqvae.to(torch.float32)  # vqvae is producing nans in fp16
pipe = pipe.to("cuda")

prompt = "a man with glasses"
input_image = (
    load_image(
        "https://raw.githubusercontent.com/huggingface/amused/main/assets/inpainting_256_orig.png"
    )
    .resize((512, 512))
    .convert("RGB")
)
mask = (
    load_image(
        "https://raw.githubusercontent.com/huggingface/amused/main/assets/inpainting_256_mask.png"
    )
    .resize((512, 512))
    .convert("L")
)    

image = pipe(prompt, input_image, mask, generator=torch.manual_seed(3)).images[0]
image.save(f"inpainting_512.png")
```
📜 Docs: https://huggingface.co/docs/diffusers/main/en/api/pipelines/amused

🛠️ Models:

- amused-256: https://huggingface.co/amused/amused-256 (603M params)
- amused-512: https://huggingface.co/amused/amused-512 (608M params)



In [ ]:
!pip install diffusers
!pip install accelerate


# 3 倍高速な SDXL

TextToImage の拡散モデルの推論レイテンシーを高速化するために使用できる一連の最適化手法を紹介します。

これらはすべて、追加の C++ コードを必要とせずにネイティブ PyTorch で実行できます。

![](https://github.com/huggingface/diffusers/assets/22957388/36d6cf13-5794-4332-b197-b07c279935a1)

これらのテクニックは Stable Diffusion XL (SDXL) に固有のものではなく、他のテキストから画像への拡散モデルを改善するためにも使用できます。 以下に提供される詳細なドキュメントを確認することをお勧めします。

📜 Docs: https://huggingface.co/docs/diffusers/main/en/tutorials/fast_diffusion


# 中断可能なパイプライン

Diffusionプロセスの中断は、ユーザーが中間結果に満足できない場合に生成プロセスを停止できるため、ディフューザーで動作する UI を構築する場合に特に便利です。 コールバックを使用してこれをパイプラインに組み込むことができます。

このコールバック関数は、pipe、i、t、および callback_kwargs の引数を取る必要があります (これは返される必要があります)。 特定のステップ数の後に拡散プロセスを停止するには、パイプラインの _interrupt 属性を True に設定します。 コールバック内に独自のカスタム停止ロジックを自由に実装することもできます。

この例では、num_inference_steps が 50 に設定されているにもかかわらず、拡散プロセスは 10 ステップ後に停止します。

```
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe.enable_model_cpu_offload()
num_inference_steps = 50

def interrupt_callback(pipe, i, t, callback_kwargs):
    stop_idx = 10
    if i == stop_idx:
        pipe._interrupt = True

    return callback_kwargs

pipe(
    "A photo of a cat",
    num_inference_steps=num_inference_steps,
    callback_on_step_end=interrupt_callback,
)
```
📜 ドキュメント: https://huggingface.co/docs/diffusers/main/en/using-diffusers/callback

LoRA トレーニングの例におけるペフト
LoRA に関して公式にサポートされているすべてのトレーニング例に peft を組み込みました。 これによりコードが大幅に簡素化され、可読性が向上します。 peft のおかげで、LoRA のトレーニングがさらに簡単になりました。

## LCM LoRA SDXL トレーニングのよりメモリに優しいバージョン

peft のベスト プラクティスを取り入れて、SDXL の LCM LoRA トレーニングをよりメモリに優しいものにしました。 そのため、2 つの UNet (教師と生徒) を初期化する必要はなくなりました。 このバージョンには、迅速な実験のためにデータセット ライブラリも統合されています。 詳細については、[このセクション](https://github.com/huggingface/diffusers/blob/main/examples/consistency_distillation/README_sdxl.md)をご覧ください。

In [ ]:
import torch
from diffusers import AmusedPipeline

pipe = AmusedPipeline.from_pretrained(
    "amused/amused-512", variant="fp16", torch_dtype=torch.float16
)
pipe.vqvae.to(torch.float32)  # vqvae is producing nans in fp16
pipe = pipe.to("cuda")

prompt = "miku"
image = pipe(prompt, generator=torch.manual_seed(39)).images[0]
image.save("miku.png")

In [ ]:
import torch
from diffusers import AmusedPipeline

# Initialize the pipeline
pipe = AmusedPipeline.from_pretrained(
    "amused/amused-512", variant="fp16", torch_dtype=torch.float16
)
pipe.vqvae.to(torch.float32)  # Fix for vqvae producing NaNs in fp16
pipe = pipe.to("cuda")

# Define your prompt
num_images = 9
prompt = "miku in dancing"
for i in range(num_images):
    image = pipe(prompt, generator=torch.manual_seed(i)).images[0]
    image.save(f"miku_in_dancing{i}.png")


In [ ]:
import torch
from diffusers import AmusedImg2ImgPipeline
from diffusers.utils import load_image

pipe = AmusedImg2ImgPipeline.from_pretrained(
    "amused/amused-512", variant="fp16", torch_dtype=torch.float16
)
pipe.vqvae.to(torch.float32)  # vqvae is producing nans in fp16
pipe = pipe.to("cuda")

prompt = "apple watercolor"
input_image = (
    load_image(
        "miku_39.png"
    )
    .resize((512, 512))
    .convert("RGB")
)

image = pipe(prompt, input_image, strength=0.7, generator=torch.manual_seed(39)).images[0]
image.save("image2image_512.png")